In [ ]:
from sklearn.model_selection import train_test_split
import seaborn as sns
import os
import shutil
%matplotlib inline

In [ ]:
path = 'output/output/dogscats/'
train_filenames = os.listdir(path+'train/')
train_cat = filter(lambda x:x[:3] == 'cat', train_filenames)
train_dog = filter(lambda x:x[:3] == 'dog', train_filenames)
x = ['train_cat', 'train_dog', 'test']
y = [len(os.listdir(path+'train/cats/')), len(os.listdir(path+'train/dogs/')), len(os.listdir(path+'test_small/cats/'))]
ax = sns.barplot(x=x, y=y)

In [ ]:
mytrain = os.listdir(path+'train/cats/')
mytrain += os.listdir(path+'train/dogs/')
myvalid = os.listdir(path+'valid/cats/')
myvalid += os.listdir(path+'valid/dogs/')

print len(mytrain), len(myvalid)

In [ ]:
mytrain_cat = filter(lambda x:x[:3] == 'cat', mytrain)
mytrain_dog = filter(lambda x:x[:3] == 'dog', mytrain)
myvalid_cat = filter(lambda x:x[:3] == 'cat', myvalid)
myvalid_dog = filter(lambda x:x[:3] == 'dog', myvalid)
x = ['mytrain_cat', 'mytrain_dog', 'myvalid_cat', 'myvalid_dog']
y = [len(mytrain_cat), len(mytrain_dog), len(myvalid_cat), len(myvalid_dog)]

ax = sns.barplot(x=x, y=y)

def remove_and_create_class(dirname):
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)
    os.mkdir(dirname+'/cats')
    os.mkdir(dirname+'/dogs')

remove_and_create_class('mytrain')
remove_and_create_class('myvalid')

for filename in mytrain_cat:
    os.symlink('output/output/dogscats/'+'train/cats/'+filename, 'mytrain/cats/'+filename)

for filename in mytrain_dog:
    os.symlink('output/output/dogscats/'+'train/dogs/'+filename, 'mytrain/dogs/'+filename)

for filename in myvalid_cat:
    os.symlink('output/output/dogscats/'+'valid/cats/'+filename, 'myvalid/cats/'+filename)

for filename in myvalid_dog:
    os.symlink('output/output/dogscats/'+'valid/dogs/'+filename, 'myvalid/dogs/'+filename)

In [ ]:
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.applications import VGG16
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, add, Input
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D, GlobalAveragePooling2D
from keras.utils import np_utils
from keras.models import model_from_json
from keras import backend as K
from keras.preprocessing import image
from keras.optimizers import SGD,Adam,Adagrad
from keras.utils.data_utils import get_file
import random
import os
import cv2
from keras import models
from keras import layers
from keras import optimizers
 

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
image_width = 224
image_height = 224
image_size = (image_width, image_height)
batch_size = 16

train_datagen = ImageDataGenerator(rescale=1.0/255)
train_generator = train_datagen.flow_from_directory(
        'mytrain',  # this is the target directory
        target_size=image_size,  # all images will be resized to 224x224
        batch_size=batch_size,
        class_mode='binary')

validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(
        'myvalid',  # this is the target directory
        target_size=image_size,  # all images will be resized to 224x224
        batch_size=batch_size,
        class_mode='binary')



In [ ]:
base_model = VGG16(input_tensor=Input((224, 224, 3)), weights='imagenet', include_top=False)

for layers in base_model.layers:
    layers.trainable = False

    
print base_model.output

In [ ]:
from keras.utils import to_categorical

train_features = np.zeros(shape=(22000, 7, 7, 512))
train_labels = np.zeros(shape=(22000,2))
batch_size = 16
i = 0

for inputs_batch, labels_batch in train_generator :
    features_batch = base_model.predict(inputs_batch)
    train_features[i * batch_size : (i + 1) * batch_size] = features_batch
    train_labels[i * batch_size : (i + 1) * batch_size] = to_categorical(labels_batch)
    i += 1
    if i * batch_size >= 22000:
        break
         

train_features = np.reshape(train_features, (22000, 7 * 7 * 512))

In [ ]:
from keras.utils import to_categorical

In [ ]:
valid_features = np.zeros(shape=(2000, 7, 7, 512))
valid_labels = np.zeros(shape=(2000,2))
batch_size = 16
i = 0

for inputs_batch, labels_batch in validation_generator :
    features_batch = base_model.predict(inputs_batch)
    valid_features[i * batch_size : (i + 1) * batch_size] = features_batch
    valid_labels[i * batch_size : (i + 1) * batch_size] = to_categorical(labels_batch)
    i += 1
    if i * batch_size >= 2000:
        break
        
         

valid_features = np.reshape(valid_features, (2000, 7 * 7 * 512))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
image_width = 224
image_height = 224
image_size = (image_width, image_height)

test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_directory(
        'mytest-lbl',  # this is the target directory
        target_size=image_size,  # all images will be resized to 224x224
        batch_size=20,
        class_mode='binary')

test_features = np.zeros(shape=(1000, 7, 7, 512))
test_labels = np.zeros(shape=(1000,))
batch_size = 20
i = 0

for inputs_batch,labels_batch in test_generator :
    features_batch = base_model.predict(inputs_batch)
    test_features[i * batch_size : (i + 1) * batch_size] = features_batch
    test_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= 1000:
        break
   

test_features = np.reshape(test_features, (1000, 7 * 7 * 512))

In [ ]:

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=7 * 7 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))

In [ ]:
initial_weights = model.get_weights()

model_json = model.to_json()
with open("model_vgg.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("initial_weights_vgg.h5")

json_file = open('model_vgg.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

In [ ]:
batch_size = 16
model0 = model_from_json(loaded_model_json)
model0.load_weights("initial_weights_vgg.h5")
model0.compile(optimizer=SGD(lr=0.002),loss='categorical_crossentropy', metrics=['accuracy'])
h=model0.fit(train_features, train_labels, batch_size=batch_size, epochs=50, validation_data=(valid_features, valid_labels))
model0.save("sgd_vgg_1.h5")
with open('file_sgd_vgg_1.json', 'w') as f:
    json.dump(h.history, f)

model1 = model_from_json(loaded_model_json)
model1.load_weights("initial_weights_vgg.h5")
model1.compile(optimizer=SGD(lr=0.001),loss='categorical_crossentropy', metrics=['accuracy'])
h1=model1.fit(train_features, train_labels, batch_size=batch_size, epochs=50, validation_data=(valid_features, valid_labels))
model1.save("sgd_vgg_2.h5")
with open('file_sgd_vgg_2.json', 'w') as f:
    json.dump(h1.history, f)

model2 = model_from_json(loaded_model_json)
model2.load_weights("initial_weights_vgg.h5")
model2.compile(optimizer=SGD(lr=0.0005),loss='categorical_crossentropy', metrics=['accuracy'])
h2=model2.fit(train_features, train_labels, batch_size=batch_size, epochs=50, validation_data=(valid_features, valid_labels))
model2.save("sgd_vgg_3.h5")
with open('file_sgd_vgg_3.json', 'w') as f:
    json.dump(h2.history, f)

In [ ]:
from keras.optimizers import Adam

model3 = model_from_json(loaded_model_json)
model3.load_weights("initial_weights_vgg.h5")
model3.compile(optimizer=Adam(lr=0.002, beta_1=0.9, beta_2=0.999, decay=0.0),loss='categorical_crossentropy', metrics=['accuracy'])
h3=model3.fit(train_features, train_labels, batch_size=batch_size, epochs=50, validation_data=(valid_features, valid_labels))
model3.save("adam_vgg_4.h5")

model4 = model_from_json(loaded_model_json)
model4.load_weights("initial_weights_vgg.h5")
model4.compile(optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=0.0),loss='categorical_crossentropy', metrics=['accuracy'])
h4=model4.fit(train_features, train_labels, batch_size=batch_size, epochs=50, validation_data=(valid_features, valid_labels))
model4.save("adam_vgg_5.h5")

model5 = model_from_json(loaded_model_json)
model5.load_weights("initial_weights_vgg.h5")
model5.compile(optimizer=Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, decay=0.0),loss='categorical_crossentropy', metrics=['accuracy'])
h5=model5.fit(train_features, train_labels, batch_size=batch_size, epochs=50, validation_data=(valid_features, valid_labels))
model5.save("adam_vgg_6.h5")

In [ ]:
from keras.optimizers import Adagrad

model6 = model_from_json(loaded_model_json)
model6.load_weights("initial_weights_vgg.h5")
model6.compile(optimizer=Adagrad(lr=0.002,  decay=0.0),loss='categorical_crossentropy', metrics=['accuracy'])
h6=model6.fit(train_features, train_labels, batch_size=batch_size, epochs=50, validation_data=(valid_features, valid_labels))
model6.save("adag_vgg_7.h5")

model7 = model_from_json(loaded_model_json)
model7.load_weights("initial_weights_vgg.h5")
model7.compile(optimizer=Adagrad(lr=0.001,  decay=0.0),loss='categorical_crossentropy', metrics=['accuracy'])
h7=model7.fit(train_features, train_labels, batch_size=batch_size, epochs=50, validation_data=(valid_features, valid_labels))
model7.save("adag_vgg_8.h5")

model8 = model_from_json(loaded_model_json)
model8.load_weights("initial_weights_vgg.h5")
model8.compile(optimizer=Adagrad(lr=0.0005,  decay=0.0),loss='categorical_crossentropy', metrics=['accuracy'])
h8=model8.fit(train_features, train_labels, batch_size=batch_size, epochs=50, validation_data=(valid_features, valid_labels))
model8.save("adag_vgg_9.h5")

### Test Error

In [ ]:
from keras.utils import to_categorical

#Available:h3-h8

#To show that parameters learnt are quantitatively different : Lead to difference in test values
#test1_labels = h.model.predict(test_features)
#test2_labels = h1.model.predict(test_features)
#test3_labels = h2.model.predict(test_features)

#test3_labels = h3.model.predict(test_features)
#test4_labels = h4.model.predict(test_features)
#test5_labels = h5.model.predict(test_features)

#test6_labels = h6.model.predict(test_features)
#test7_labels = h7.model.predict(test_features)
#test8_labels = h8.model.predict(test_features)

test_actual = to_categorical(test_labels)


#print np.mean((test1_labels - test_actual)**2)
#print np.mean((test2_labels - test_actual)**2)
#print np.mean((test3_labels - test_actual)**2)


#print np.mean((test3_labels - test_actual)**2)
#print np.mean((test4_labels - test_actual)**2)
#print np.mean((test5_labels - test_actual)**2)

#print np.mean((test6_labels - test_actual)**2)
#print np.mean((test7_labels - test_actual)**2)
#print np.mean((test8_labels - test_actual)**2)

In [ ]:
h.model.evaluate(test_features,test_actual)

### Plot Training Error

In [ ]:
#Plot Training Error
#trn_loss_run1 = [h.history['acc'][-1],h3.history['acc'][-1],h6.history['acc'][-1]]
#trn_loss_run2 = [h1.history['acc'][-1],h4.history['acc'][-1],h7.history['acc'][-1]]
#trn_loss_run3 = [h2.history['acc'][-1],h5.history['acc'][-1],h8.history['acc'][-1]]

trn_loss_run1 = [0.9892,0.9741,0.9804]
trn_loss_run2 = [0.9876,0.9817,0.9988]
trn_loss_run3 = [0.9784,0.9882,0.9941]
labels = ['SGD', 'Adam', 'Adagrad']

plt.figure(figsize=(5, 3))  
plt.scatter(labels,trn_loss_run1)
plt.scatter(labels,trn_loss_run2)
plt.scatter(labels,trn_loss_run3)
plt.title('Training Accuracy')
plt.ylabel('trn_acc')

### Plot Testing Error

In [ ]:
tst_loss_run1 = [model0.evaluate(test_features,test_actual)[1],model3.evaluate(test_features,test_actual)[1],model6.evaluate(test_features,test_actual)[1]]
tst_loss_run2 = [model1.evaluate(test_features,test_actual)[1],model4.evaluate(test_features,test_actual)[1], model7.evaluate(test_features,test_actual)[1]]
tst_loss_run3 = [model2.evaluate(test_features,test_actual)[1],model5.evaluate(test_features,test_actual)[1],model8.evaluate(test_features,test_actual)[1]]
labels = ['SGD', 'Adam', 'Adagrad']

plt.figure(figsize=(5, 3))  
plt.scatter(labels,tst_loss_run1)
plt.scatter(labels,tst_loss_run2)
plt.scatter(labels,tst_loss_run3)
plt.title('Testing Accuracy for VGG16')
plt.ylabel('test_acc')
plt.yticks(np.arange(0.915, 0.96, 0.005))

plt.show()

### To show that parameters learnt are quantitatively different : Lead to difference in test values

In [ ]:
from keras.models import load_model

model0 = load_model('sgd_vgg_1.h5')
model3 = load_model('adam_vgg_4.h5')
model6 = load_model('adag_vgg_7.h5')


test0_labels = model0.predict(test_features)
test3_labels = model3.predict(test_features)
test6_labels = model6.predict(test_features)




In [ ]:
model1 = load_model('sgd_vgg_2.h5')
model4 = load_model('adam_vgg_5.h5')
model7 = load_model('adag_vgg_8.h5')



test1_labels = model1.predict(test_features)
test4_labels = model4.predict(test_features)
test7_labels = model7.predict(test_features)



In [ ]:
model2 = load_model('sgd_vgg_3.h5')
model5 = load_model('adam_vgg_6.h5')
model8 = load_model('adag_vgg_9.h5')

test2_labels = model2.predict(test_features)
test5_labels = model5.predict(test_features)
test8_labels = model8.predict(test_features)




In [ ]:
json_file = open('model_vgg.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

In [ ]:
dist_loss_sgd_adam = [np.linalg.norm((test0_labels - test3_labels)),np.linalg.norm((test0_labels - test6_labels)),np.linalg.norm((test3_labels - test6_labels)) ]
dist_loss_sgd_adag = [np.linalg.norm((test1_labels - test4_labels)),np.linalg.norm((test1_labels - test7_labels)),np.linalg.norm((test4_labels - test7_labels)) ]
dist_loss_adag_adam =[np.linalg.norm((test2_labels - test5_labels)),np.linalg.norm((test2_labels - test8_labels)),np.linalg.norm((test5_labels - test8_labels)) ]
labels = ['SGD-Adam', 'SGD-Adagrad', 'Adam-Adagrad']

plt.figure(figsize=(10, 6))  
plt.scatter(labels,dist_loss_sgd_adam)
plt.scatter(labels,dist_loss_sgd_adag)
plt.scatter(labels,dist_loss_adag_adam)
plt.title('Distance between weights for different optimizers')
plt.ylabel('distance')

In [ ]:
tst_loss_run1 = [h.model.evaluate(test_features,test_actual)[1],h3.model.evaluate(test_features,test_actual)[1],h6.model.evaluate(test_features,test_actual)[1]]
tst_loss_run2 = [h1.model.evaluate(test_features,test_actual)[1],h4.model.evaluate(test_features,test_actual)[1], h7.model.evaluate(test_features,test_actual)[1]]
tst_loss_run3 = [h2.model.evaluate(test_features,test_actual)[1],h5.model.evaluate(test_features,test_actual)[1],h8.model.evaluate(test_features,test_actual)[1]]
labels = ['SGD', 'Adam', 'Adagrad']

plt.figure(figsize=(5, 3))  
plt.scatter(labels,tst_loss_run1)
plt.scatter(labels,tst_loss_run2)
plt.scatter(labels,tst_loss_run3)
plt.title('Testing Accuracy for VGG16')
plt.ylabel('test_acc')

In [ ]:
json_file = open('model_vgg.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

In [ ]:
#To show that parameters learnt are quantitatively different : Lead to difference in test values
#SGD-vs-Adam
alpha = np.arange(0, 1.01, 0.01)
loss1 = np.zeros((len(alpha),))
loss2 = np.zeros((len(alpha),))
loss3 = np.zeros((len(alpha),))

i=0
for a in alpha:
    model13 = model_from_json(loaded_model_json)
    f = [a * w for w in model0.get_weights()]
    s = [(1-a) * w for w in model3.get_weights()]
    model13.set_weights([x+y for x,y in zip(f,s)])
    model13.compile(optimizer=Adam(lr=0.002, beta_1=0.9, beta_2=0.999, decay=0.0),loss='categorical_crossentropy', metrics=['accuracy'])
    loss1[i] = model13.evaluate(test_features,test_actual)[0]
    i +=1

    
i=0    
for a in alpha:
    model13 = model_from_json(loaded_model_json)
    f = [a * w for w in model1.get_weights()]
    s = [(1-a) * w for w in model4.get_weights()]
    model13.set_weights([x+y for x,y in zip(f,s)])
    model13.compile(optimizer=Adam(lr=0.002, beta_1=0.9, beta_2=0.999, decay=0.0),loss='categorical_crossentropy', metrics=['accuracy'])
    loss2[i] = model13.evaluate(test_features,test_actual)[0]
    i +=1

i=0    
for a in alpha:
    model13 = model_from_json(loaded_model_json)
    f = [a * w for w in model2.get_weights()]
    s = [(1-a) * w for w in model5.get_weights()]
    model13.set_weights([x+y for x,y in zip(f,s)])
    model13.compile(optimizer=Adam(lr=0.002, beta_1=0.9, beta_2=0.999, decay=0.0),loss='categorical_crossentropy', metrics=['accuracy'])
    loss3[i] = model13.evaluate(test_features,test_actual)[0]
    i +=1




plt.figure(figsize=(10, 6))  
plt.plot(alpha,loss1,'r--',alpha,loss2,'b--',alpha,loss3,'g--')
plt.legend(['(p*sgd + (1-p)*adam)'], loc='upper right')  
plt.ylabel("loss")
plt.xlabel("p")
plt.show()  

In [ ]:
plt.plot(alpha,loss1)


In [ ]:
#Adam-vs-Adagrad
#Adam-vs-Adagrad

alpha = np.arange(0, 1.05, 0.05)
loss1 = np.zeros((len(alpha),))

i=0
for a in alpha:
    model36 = model_from_json(loaded_model_json)
    f = [a * w for w in model3.get_weights()]
    s = [(1-a) * w for w in model6.get_weights()]
    model36.set_weights([x+y for x,y in zip(f,s)])
    #t36_labels = model36.predict(test_features)
    model36.compile(optimizer=Adam(lr=0.002, beta_1=0.9, beta_2=0.999, decay=0.0),loss='categorical_crossentropy', metrics=['accuracy'])
    loss1[i] = model36.evaluate(test_features,test_actual)[0]
    i +=1


    
loss2 = np.zeros((len(alpha),))
loss3 = np.zeros((len(alpha),))

i=0
for a in alpha:
    model36 = model_from_json(loaded_model_json)
    f = [a * w for w in model4.get_weights()]
    s = [(1-a) * w for w in model7.get_weights()]
    model36.set_weights([x+y for x,y in zip(f,s)])
    #t36_labels = model36.predict(test_features)
    model36.compile(optimizer=Adam(lr=0.002, beta_1=0.9, beta_2=0.999, decay=0.0),loss='categorical_crossentropy', metrics=['accuracy'])
    loss2[i] = model36.evaluate(test_features,test_actual)[0]
    i +=1
    
    
    
i=0
for a in alpha:
    model36 = model_from_json(loaded_model_json)
    f = [a * w for w in model5.get_weights()]
    s = [(1-a) * w for w in model8.get_weights()]
    model36.set_weights([x+y for x,y in zip(f,s)])
    #t36_labels = model36.predict(test_features)
    model36.compile(optimizer=Adam(lr=0.002, beta_1=0.9, beta_2=0.999, decay=0.0),loss='categorical_crossentropy', metrics=['accuracy'])
    loss3[i] = model36.evaluate(test_features,test_actual)[0]
    i +=1


plt.figure(figsize=(10, 6))  
plt.plot(alpha,loss1,'r--',alpha,loss2,'b--',alpha,loss3,'g--')
plt.legend(['(p*adam + (1-p)*adagrad)'], loc='upper right')  
plt.ylabel("loss")
plt.xlabel("p")
plt.show()


In [ ]:
#SGD-vs-Adagrad

alpha = np.arange(0, 1.01, 0.01)
loss1 = np.zeros((len(alpha),))
loss2 = np.zeros((len(alpha),))
loss3 = np.zeros((len(alpha),))
i=0
for a in alpha:
    model16 = model_from_json(loaded_model_json)
    f = [a * w for w in model0.get_weights()]
    s = [(1-a) * w for w in model6.get_weights()]
    model16.set_weights([x+y for x,y in zip(f,s)])
    #t16_labels = model16.predict(test_features)
    model16.compile(optimizer=Adam(lr=0.002, beta_1=0.9, beta_2=0.999, decay=0.0),loss='categorical_crossentropy', metrics=['accuracy'])
    loss1[i] = model16.evaluate(test_features,test_actual)[0]
    
    i +=1
    
i=0    
for a in alpha:
    model16 = model_from_json(loaded_model_json)
    f = [a * w for w in model1.get_weights()]
    s = [(1-a) * w for w in model7.get_weights()]
    model16.set_weights([x+y for x,y in zip(f,s)])
    #t16_labels = model16.predict(test_features)
    model16.compile(optimizer=Adam(lr=0.002, beta_1=0.9, beta_2=0.999, decay=0.0),loss='categorical_crossentropy', metrics=['accuracy'])
    loss2[i] = model16.evaluate(test_features,test_actual)[0]
    
    i +=1

i=0
for a in alpha:
    model16 = model_from_json(loaded_model_json)
    f = [a * w for w in model2.get_weights()]
    s = [(1-a) * w for w in model8.get_weights()]
    model16.set_weights([x+y for x,y in zip(f,s)])
    #t16_labels = model16.predict(test_features)
    model16.compile(optimizer=Adam(lr=0.002, beta_1=0.9, beta_2=0.999, decay=0.0),loss='categorical_crossentropy', metrics=['accuracy'])
    loss3[i] = model16.evaluate(test_features,test_actual)[0]
    
    i +=1





plt.figure(figsize=(10, 6))  
plt.plot(alpha,loss1,'r--',alpha,loss2,'b--',alpha,loss3,'g--')
plt.legend(['(p*sgd + (1-p)*adagrad)'], loc='upper right')  
plt.ylabel("loss")
plt.xlabel("p")
plt.show()


In [ ]:
model16 = model_from_json(loaded_model_json)
f = [0.4 * w for w in model0.get_weights()]
s = [(0.6) * w for w in model6.get_weights()]
model16.set_weights([x+y for x,y in zip(f,s)])
#t16_labels = model16.predict(test_features)
model16.compile(optimizer=Adam(lr=0.002, beta_1=0.9, beta_2=0.999, decay=0.0),loss='categorical_crossentropy', metrics=['accuracy'])
loss_smaller_test = model16.evaluate(test_features,test_actual)[0]
loss_smaller_valid = model16.evaluate(valid_features,valid_labels)[0]

In [ ]:
loss_smaller_test

In [ ]:
#SGD-vs-Adam

alpha = np.arange(0, 1.05, 0.05)
loss1 = np.zeros((len(alpha),))
loss2 = np.zeros((len(alpha),))
loss3 = np.zeros((len(alpha),))
loss4 = np.zeros((len(alpha),))
loss5 = np.zeros((len(alpha),))
loss6 = np.zeros((len(alpha),))

i=0
for a in alpha:
    model16 = model_from_json(loaded_model_json)
    f = [a * w for w in model0.get_weights()]
    s = [(1-a) * w for w in model3.get_weights()]
    model16.set_weights([x+y for x,y in zip(f,s)])
    t16_labels = model16.predict(test_features)
    loss1[i] = np.linalg.norm(t16_labels - test0_labels)
    loss2[i] = np.linalg.norm(t16_labels - test3_labels)
    i +=1

    

###########
i=0
for a in alpha:
    model16 = model_from_json(loaded_model_json)
    f = [a * w for w in model1.get_weights()]
    s = [(1-a) * w for w in model4.get_weights()]
    model16.set_weights([x+y for x,y in zip(f,s)])
    t16_labels = model16.predict(test_features)
    loss3[i] = np.linalg.norm(t16_labels - test1_labels)
    loss4[i] = np.linalg.norm(t16_labels - test4_labels)
    i +=1



################
i=0
for a in alpha:
    model16 = model_from_json(loaded_model_json)
    f = [a * w for w in model2.get_weights()]
    s = [(1-a) * w for w in model5.get_weights()]
    model16.set_weights([x+y for x,y in zip(f,s)])
    t16_labels = model16.predict(test_features)
    loss5[i] = np.linalg.norm(t16_labels - test2_labels)
    loss6[i] = np.linalg.norm(t16_labels - test5_labels)
    i +=1


plt.figure(figsize=(10, 6))  
plt.plot(alpha,loss1,'r--',alpha,loss2,'r:',alpha,loss3,'b--',alpha,loss4,'b:',alpha,loss5,'g--',alpha,loss6,'g:')
plt.legend(['sgd-vs-(p*sgd + (1-p)*adam)', 'adam-vs-(p*sgd + (1-p)*adam)'], loc='upper right') 
plt.ylabel("distance")
plt.xlabel("p")
plt.show()

In [ ]:
#Adam-vs-Adagrad

alpha = np.arange(0, 1.05, 0.05)
loss1 = np.zeros((len(alpha),))
loss2 = np.zeros((len(alpha),))
loss3 = np.zeros((len(alpha),))
loss4 = np.zeros((len(alpha),))
loss5 = np.zeros((len(alpha),))
loss6 = np.zeros((len(alpha),))
i=0
for a in alpha:
    model16 = model_from_json(loaded_model_json)
    f = [a * w for w in model3.get_weights()]
    s = [(1-a) * w for w in model6.get_weights()]
    model16.set_weights([x+y for x,y in zip(f,s)])
    t16_labels = model16.predict(test_features)
    loss1[i] = np.linalg.norm(t16_labels - test3_labels)
    loss2[i] = np.linalg.norm(t16_labels - test6_labels)
    i +=1


###########
i=0
for a in alpha:
    model16 = model_from_json(loaded_model_json)
    f = [a * w for w in model4.get_weights()]
    s = [(1-a) * w for w in model7.get_weights()]
    model16.set_weights([x+y for x,y in zip(f,s)])
    t16_labels = model16.predict(test_features)
    loss3[i] = np.linalg.norm(t16_labels - test4_labels)
    loss4[i] = np.linalg.norm(t16_labels - test7_labels)
    i +=1

################
i=0
for a in alpha:
    model16 = model_from_json(loaded_model_json)
    f = [a * w for w in model5.get_weights()]
    s = [(1-a) * w for w in model8.get_weights()]
    model16.set_weights([x+y for x,y in zip(f,s)])
    t16_labels = model16.predict(test_features)
    loss5[i] = np.linalg.norm(t16_labels - test5_labels)
    loss6[i] = np.linalg.norm(t16_labels - test8_labels)
    i +=1


plt.figure(figsize=(10, 6))  
plt.plot(alpha,loss1,'r--',alpha,loss2,'r:',alpha,loss3,'b--',alpha,loss4,'b:',alpha,loss5,'g--',alpha,loss6,'g:')
plt.legend(['adam-vs-(p*adam + (1-p)*adagrad)', 'adagrad-vs-(p*adam + (1-p)*adagrad)'], loc='upper right') 
plt.ylabel("distance")
plt.xlabel("p")
plt.show()








In [ ]:
plt.figure(figsize=(10, 6))  
plt.plot(alpha,loss5,'g--',alpha,loss6,'g:')
plt.legend(['adam-vs-(p*adam + (1-p)*adagrad)', 'adagrad-vs-(p*adam + (1-p)*adagrad)'], loc='upper right') 
plt.ylabel("distance")
plt.xlabel("p")
plt.show()



In [ ]:
#SGD-vs-Adagrad

alpha = np.arange(0, 1.05, 0.05)
loss1 = np.zeros((len(alpha),))
loss2 = np.zeros((len(alpha),))
loss3 = np.zeros((len(alpha),))
loss4 = np.zeros((len(alpha),))
loss5 = np.zeros((len(alpha),))
loss6 = np.zeros((len(alpha),))


i=0
for a in alpha:
    model16 = model_from_json(loaded_model_json)
    f = [a * w for w in model0.get_weights()]
    s = [(1-a) * w for w in model6.get_weights()]
    model16.set_weights([x+y for x,y in zip(f,s)])
    t16_labels = model16.predict(test_features)
    loss1[i] = np.linalg.norm(t16_labels - test0_labels)
    loss2[i] = np.linalg.norm(t16_labels - test6_labels)
    i +=1




###########
i=0
for a in alpha:
    model16 = model_from_json(loaded_model_json)
    f = [a * w for w in model1.get_weights()]
    s = [(1-a) * w for w in model7.get_weights()]
    model16.set_weights([x+y for x,y in zip(f,s)])
    t16_labels = model16.predict(test_features)
    loss3[i] = np.linalg.norm(t16_labels - test1_labels)
    loss4[i] = np.linalg.norm(t16_labels - test7_labels)
    i +=1



################
i=0
for a in alpha:
    model16 = model_from_json(loaded_model_json)
    f = [a * w for w in model2.get_weights()]
    s = [(1-a) * w for w in model8.get_weights()]
    model16.set_weights([x+y for x,y in zip(f,s)])
    t16_labels = model16.predict(test_features)
    loss5[i] = np.linalg.norm(t16_labels - test2_labels)
    loss6[i] = np.linalg.norm(t16_labels - test8_labels)
    i +=1


plt.figure(figsize=(10, 6))  
plt.plot(alpha,loss1,'r--',alpha,loss2,'r:',alpha,loss3,'b--',alpha,loss4,'b:',alpha,loss5,'g--',alpha,loss6,'g:')
plt.legend(['sgd-vs-(p*sgd + (1-p)*adagrad)', 'adagrad-vs-(p*sgd + (1-p)*adagrad)'], loc='upper right') 
plt.ylabel("distance")
plt.xlabel("p")
plt.show()





